<a href="https://colab.research.google.com/github/NaamaSchweitzer/CV-waste-classification/blob/ModelProcessing/YoloV8n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O CV_Waste_Classification.ipynb \
  https://raw.githubusercontent.com/NaamaSchweitzer/CV-waste-classification/Preprocessing/CV_Waste_Classification.ipynb

%run CV_Waste_Classification.ipynb

--2025-11-29 12:45:05--  https://raw.githubusercontent.com/NaamaSchweitzer/CV-waste-classification/Preprocessing/CV_Waste_Classification.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116592 (114K) [text/plain]
Saving to: ‘CV_Waste_Classification.ipynb’

CV_Waste_Classifica 100%[===================>] 113.86K  --.-KB/s    in 0.01s   

2025-11-29 12:45:05 (10.9 MB/s) - ‘CV_Waste_Classification.ipynb’ saved [116592/116592]

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Kaggle authentication setup complete.
Dataset URL: https://www.kaggle.com/datasets/alyyan/trash-detection
License(s): MIT
100% 1.23G/1.23G [00:16<00:00, 79.1MB/s]
100% 1.23G/1.23G [00:16<00:00, 82.0MB/s]

## YOLOv8 (Ultralytics)


In [2]:
!pip install ultralytics
# ===== IMPORTANT: RUN THIS CELL FIRST =====
import os
from ultralytics import YOLO

# Set the CORRECT path to data.yaml
data_config_path = './trash-detection/Dataset/data.yaml'

# Verify it exists
if os.path.exists(data_config_path):
    print(f"✓ Data configuration found at {data_config_path}")
else:
    print(f"✗ Error: data.yaml not found at {data_config_path}")

# Set model name
model_name = 'yolov8n.pt'

# Helper function
def load_pretrained_model(name):
    return YOLO(name)

print(f"✓ Model name set to: {model_name}")
print(f"✓ data_config_path = {data_config_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✓ Data configuration found at ./trash-detection/Dataset/data.yaml
✓ Model name set to: yolov8n.pt
✓ data_config_path = ./trash-detection/Dataset/data.yaml


Installation and Environment Setup
Before we begin, we need to install the ultralytics library. This library provides all the tools needed for training, evaluating, and using YOLOv8 models. The following cell installs the library, imports the required modules, and checks the environment (including GPU detection).

In [3]:
import ultralytics
from ultralytics import YOLO
import torch
import os
import glob
import random
import cv2
import matplotlib.pyplot as plt
from IPython.display import display, Image

ultralytics.checks()

print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Warning: GPU not detected. Training might be slow.")

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.9/112.6 GB disk)

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4


##Selecting a Base Model (Pretrained Backbone)
Model Selection and Transfer Learning
I chose the YOLOv8n (Nano version) model as our pretrained backbone. This is a lightweight and efficient model pre-trained on the large COCO dataset. Loading these weights gives us a strong foundation for Transfer Learning. We also verify that the path to the data.yaml file is correct.

In [5]:
import os
import yaml
import kagglehub
from ultralytics import YOLO

# Step 1: Download dataset
print("Downloading dataset...")
path = "./trash-detection"
dataset_root = f"{path}/Dataset"
print(f"Dataset root: {dataset_root}")

# Step 2: Read and fix data.yaml
original_yaml_path = f"{dataset_root}/data.yaml"
print(f"\nOriginal data.yaml contents:")
with open(original_yaml_path, 'r') as f:
    content = f.read()
    print(content)

# Step 4: Verify images exist
train_path = f"{dataset_root}/images/augmented_train"
val_path = f"{dataset_root}/images/new_val"
test_path = f"{dataset_root}/images/test"
print(f"\nVerifying paths:")
print(f"  Train images exist: {os.path.exists(train_path)} ({len(os.listdir(train_path)) if os.path.exists(train_path) else 0} files)")
print(f"  Val images exist: {os.path.exists(val_path)} ({len(os.listdir(val_path)) if os.path.exists(val_path) else 0} files)")
print(f"  Train images exist: {os.path.exists(test_path)} ({len(os.listdir(test_path)) if os.path.exists(test_path) else 0} files)")

Dataset root: ./trash-detection/Dataset

Original data.yaml contents:
names:
- dirt
- liquid
- marks
- trash
nc: 4
test: images/test
train: images/augmented_train
val: images/new_val


Verifying paths:
  Train images exist: True (1653 files)
  Val images exist: True (92 files)
  Train images exist: True (216 files)


## Model Training - First Run (Fine-Tuning Run 1)
Now we'll start the Fine-Tuning process. I decided to train for 30 epochs. In this run, we'll use the standard YOLOv8 parameters (including the default initial learning rate lr0 of 0.01). The results will be saved in a dedicated folder (YOLOv8n_Run1_DefaultLR) for comparison later. We use batch=-1 to allow the library to choose the optimal batch size for the available GPU memory

In [6]:
# Step 5: Train with corrected data.yaml
EPOCHS = 30
IMAGE_SIZE = 640
PROJECT_NAME = 'runs/detect'
RUN1_NAME = 'YOLOv8n_Run1_DefaultLR'

print(f"\nStarting Training Run 1 ({RUN1_NAME}) for {EPOCHS} epochs...")
print(f"Using data config: {original_yaml_path}")

yoloV8N = YOLO('yolov8n.pt')

results_run1 = yoloV8N.train(
    data=original_yaml_path,  # Use the corrected file
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    patience=10,
    batch=-1,
    project=PROJECT_NAME,
    name=RUN1_NAME
)
print("Training Run 1 finished successfully.")


Starting Training Run 1 (YOLOv8n_Run1_DefaultLR) for 30 epochs...
Using data config: ./trash-detection/Dataset/data.yaml
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./trash-detection/Dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=YOLOv8n_R